In [ ]:
from ngsolve import *
from ngsolve.meshes import Make1DMesh
from ngsolve.la import EigenValues_Preconditioner
from ngsolve.webgui import Draw
import numpy as np
import math
import scipy.linalg
from scipy import random
import scipy.sparse as sp

In [ ]:
mesh = Make1DMesh(40)
# Draw(mesh)

In [ ]:
# u = H1(mesh, order=2, dirichlet="left|right")
fes = H1(mesh, order=2, dirichlet="left|right")
u = fes.TrialFunction()
v = fes.TestFunction()

a = BilinearForm(fes,symmetric=True)
a += grad(u)*grad(v)*dx
# pre = Preconditioner(a, "bddc")

m = BilinearForm(fes,symmetric=True)
m += u*v*dx

a.Assemble()
m.Assemble()
u = GridFunction(fes)

In [ ]:
with TaskManager():
    lam = ArnoldiSolver(a.mat, m.mat, fes.FreeDofs(),
                        list(u.vecs), shift=400)

In [ ]:
print ("lam: ", lam)

In [ ]:
# rows,cols,vals = a.mat.COO()
# AS = sp.csr_matrix((vals,(rows,cols)))

In [ ]:
# rows,cols,vals = m.mat.COO()
# MS = sp.csr_matrix((vals,(rows,cols)))

In [ ]:
# eval_,evec = scipy.linalg.eig(AS.todense(),MS.todense())
# print (eval_)
# evr = eval_.real

# print(1/evr)
# print(min(abs(1/evr)))

In [ ]:
r = u.vec.CreateVector()
w = u.vec.CreateVector()
Mu = u.vec.CreateVector()
Au = u.vec.CreateVector()
Mw = u.vec.CreateVector()
Aw = u.vec.CreateVector()

In [ ]:
# r.FV().NumPy()[:] = random.rand(fes.ndof)
r.SetRandom()
# r.FV().NumPy()[:] = np.ones(fes.ndof)
u.vec.data = Projector(fes.FreeDofs(), True) * r

In [ ]:
for i in range(20):
    Au.data = a.mat * u.vec
    Mu.data = m.mat * u.vec
    auu = InnerProduct(Au, u.vec)
    muu = InnerProduct(Mu, u.vec)
    # Rayleigh quotient
    lam = auu/muu
    print (lam )
    # residual
    r.data = Au - lam * Mu
    w.data = pre.mat * r.data
    w.data = 1/Norm(w) * w
    Aw.data = a.mat * w
    Mw.data = m.mat * w

    # setup and solve 2x2 small eigenvalue problem
    asmall = Matrix(2,2)
    asmall[0,0] = auu
    asmall[0,1] = asmall[1,0] = InnerProduct(Au, w)
    asmall[1,1] = InnerProduct(Aw, w)
    msmall = Matrix(2,2)
    msmall[0,0] = muu
    msmall[0,1] = msmall[1,0] = InnerProduct(Mu, w)
    msmall[1,1] = InnerProduct(Mw, w)
    # print ("asmall =", asmall, ", msmall = ", msmall)


    eval_,evec = scipy.linalg.eigh(a=asmall, b=msmall)
    # print (eval, evec)
    u.vec.data = float(evec[0,0]) * u.vec + float(evec[1,0]) * w
    print (eval_)
    


In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve.meshes import Make1DMesh
import math

In [2]:
mesh = Make1DMesh(100)

In [6]:
fes = H1(mesh, order=2, dirichlet="left|right")
u = fes.TrialFunction()
v = fes.TestFunction()

a = BilinearForm(fes)
a += grad(u)*grad(v)*dx


m = BilinearForm(fes)
m += u*v*dx

a.Assemble()
m.Assemble()

u = GridFunction(fes,multidim=3)

In [7]:
with TaskManager():
    lam = ArnoldiSolver(a.mat, m.mat, fes.FreeDofs(),list(u.vecs), shift=1)

In [8]:
print ("lam: ", lam)

lam:   (9.8696,0)
 (39.4784,0)
 (88.8272,0)

